In [ ]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.2 MB/s 
     |████████████████████████████████| 9.3 MB 34.8 MB/s 
     |████████████████████████████████| 1.6 MB 40.0 MB/s 
     |████████████████████████████████| 108 kB 55.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [ ]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [ ]:
df.drop(['packer_type'],axis=1,inplace=True)

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [ ]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [ ]:
df.drop(['index'],axis=1,inplace=True)

In [ ]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.179801,0.000000,118784,4.978934,0,1
1,144,3,4,65535,184,248,7,1,0,1,...,1,4,3,0,0.000000,5.174961,99328,5.144616,0,1
2,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.460165,0.000000,93184,7.663134,1,1
3,144,3,4,65535,184,184,21,1,1,1,...,1,17,4,0,6.021402,3.002913,458240,7.643761,1,1
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.199886,1.090059,16384,5.506546,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,5.750709,2.587881,21008,6.519565,1,0
5206,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,6.582133,4.864012,909312,6.559462,1,0
5207,144,3,4,65535,184,224,7,1,0,1,...,1,5,2,0,0.000000,5.876058,218112,7.177210,0,1
5208,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,5.761412,2.267591,143360,7.486989,1,1


In [ ]:
df_5percent=df[0:5210]

In [ ]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.179801,0.000000,118784,4.978934,0,1
1,144,3,4,65535,184,248,7,1,0,1,...,1,4,3,0,0.000000,5.174961,99328,5.144616,0,1
2,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.460165,0.000000,93184,7.663134,1,1
3,144,3,4,65535,184,184,21,1,1,1,...,1,17,4,0,6.021402,3.002913,458240,7.643761,1,1
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.199886,1.090059,16384,5.506546,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,5.750709,2.587881,21008,6.519565,1,0
5206,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,6.582133,4.864012,909312,6.559462,1,0
5207,144,3,4,65535,184,224,7,1,0,1,...,1,5,2,0,0.000000,5.876058,218112,7.177210,0,1
5208,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,5.761412,2.267591,143360,7.486989,1,1


In [ ]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [ ]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [ ]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     184.0               3.0   
1      144.0   3.0        4.0     65535.0  184.0     248.0               7.0   
2      144.0   3.0        4.0     65535.0  184.0     232.0               6.0   
3      144.0   3.0        4.0     65535.0  184.0     184.0              21.0   
4      144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
5205   144.0   3.0        4.0     65535.0  184.0     264.0               5.0   
5206   144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
5207   144.0   3.0        4.0     65535.0  184.0     224.0               7.0   
5208   144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
5209   144.0   3.0        4.0     65535.0  184.0     224.0               5.0   

      C

In [ ]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.0,...,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5.210000e+03,5210.000000,5210.000000,5210.000000
mean,152.658733,10.633589,8.903263,65137.618234,202.530902,222.833013,4.662956,0.990403,0.362764,1.0,...,0.999040,1.360845,3.302111,0.156430,4.939586,2.523549,7.875718e+05,6.364756,0.544146,0.522457
std,616.499070,392.745064,251.685103,5005.606042,968.625098,48.591497,1.922882,0.097502,0.480844,0.0,...,0.030967,1.623972,1.146956,0.363297,2.521787,2.685909,5.293440e+06,1.119881,0.498095,0.499543
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.886006,0.000000,6.144000e+04,5.695785,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.148211,1.555973,1.218560e+05,6.390618,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.507072,4.685836,3.055480e+05,7.314531,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [ ]:

# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [ ]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [ ]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [ ]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      53.75
1.0      46.25
dtype: float64

In [ ]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      62.5
0.0      37.5
dtype: float64

In [ ]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [ ]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [ ]:
circuit(weights_init, data[0])

tensor(0.05142258, requires_grad=True)

In [ ]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [ ]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [ ]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.11157294, requires_grad=False), tensor(-0.17396195, requires_grad=False)], Y =  1
X = [tensor(-0.1255953, requires_grad=False), tensor(-0.17385266, requires_grad=False)], Y =  1
X = [tensor(-0.10546532, requires_grad=False), tensor(-0.17373581, requires_grad=False)], Y =  1
X = [tensor(-0.06520403, requires_grad=False), tensor(-0.17645533, requires_grad=False)], Y = -1
X = [tensor(-0.10947238, requires_grad=False), tensor(-0.17436468, requires_grad=False)], Y = -1


In [ ]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [ ]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1282470 | Accuracy: 0.4750000 
New best
Iter:     2 | Cost: 0.9595475 | Accuracy: 0.5750000 
Iter:     3 | Cost: 1.0167327 | Accuracy: 0.5625000 
Iter:     4 | Cost: 0.9812928 | Accuracy: 0.5625000 
Iter:     5 | Cost: 0.9676813 | Accuracy: 0.5625000 
New best
Iter:     6 | Cost: 0.9634823 | Accuracy: 0.5812500 
Iter:     7 | Cost: 0.9776040 | Accuracy: 0.5250000 
Iter:     8 | Cost: 1.0061256 | Accuracy: 0.5000000 
Iter:     9 | Cost: 1.0203278 | Accuracy: 0.4937500 
Iter:    10 | Cost: 1.0380028 | Accuracy: 0.4812500 
Iter:    11 | Cost: 1.0269246 | Accuracy: 0.4812500 
Iter:    12 | Cost: 0.9849232 | Accuracy: 0.5125000 
Iter:    13 | Cost: 0.9556917 | Accuracy: 0.5750000 
Iter:    14 | Cost: 0.9700213 | Accuracy: 0.5125000 
Iter:    15 | Cost: 0.9723490 | Accuracy: 0.5062500 
Iter:    16 | Cost: 1.0080490 | Accuracy: 0.4937500 
Iter:    17 | Cost: 1.0643878 | Accuracy: 0.4812500 
Iter:    18 | Cost: 1.1321698 | Accuracy: 0.4812500 
Iter:    19 | Cost:

In [ ]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [ ]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.8580880154977122, Accuracy: 68.0%


In [ ]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,1.0
2,1.0,-1.0
3,1.0,1.0
4,-1.0,-1.0
5,1.0,1.0
6,1.0,-1.0
7,1.0,-1.0
8,1.0,1.0
9,1.0,-1.0


In [ ]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.13      1.00      0.24         2
         1.0       1.00      0.66      0.79        38

    accuracy                           0.68        40
   macro avg       0.57      0.83      0.51        40
weighted avg       0.96      0.68      0.77        40

1.0
0.6578947368421053
0.7936507936507937
0.8289473684210527
